In [72]:
import numpy as np
from numpy import nan
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None 
from sklearn.preprocessing import OneHotEncoder

import nltk
import re
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial import distance

In [73]:
rel_que = pd.read_csv('data/relevant_questions.csv')
ans = pd.read_csv('data/keyword_answer.csv')
kdf = pd.read_csv('data/keywords.csv')

In [74]:
df1 = pd.read_csv('data/original_data1.csv')
df2 = pd.read_csv('data/original_data2.csv')
df3 = pd.read_csv('data/original_data3.csv')
df4 = pd.read_csv('data/original_data4.csv')
df5 = pd.read_csv('data/original_data5.csv')
df = pd.concat([df1,df2,df3,df4,df5], axis=0)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
from bs4 import BeautifulSoup
df.drop(columns=['answers', 'tags'], axis=1, inplace = True)

In [75]:
def length_text(text):
    if(type(text) != type(0.0)):
        text = text.split(' ')
        return len(text)
    else:
        return 0
ans['ans length'] = ans['answer'].apply(length_text)

In [76]:
rel_ans = pd.DataFrame()
for i in enumerate(rel_que['id']):
    rel_ans = pd.concat([ans[ans['id'] == i[1]], rel_ans])

In [77]:
topics = ['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6', 'Topic 7']

In [78]:
def get_dataframe(topic_list):
    if(type(topic_list) == type(0.0)):
        return pd.Series([0,0,0,0,0,0,0,0])
    else:
        topic_list = topic_list.split(', ')
        tl = []
        for topic in topics:
            if(topic in topic_list):
                val = 1
            else: 
                val = 0
            tl.append(val)
        return pd.Series(tl)

In [79]:
rel_ans[topics] = rel_ans['topic list'].apply(get_dataframe)
rel_ans.drop(columns=['topic list', 'id'], inplace = True)
rel_ans['answer list'] = rel_ans['answer'] + rel_ans['code']

In [80]:
que_ans = pd.DataFrame()
for i in enumerate(rel_que['id']):
    que_ans = pd.concat([df[df['id'] == i[1]], que_ans])

In [81]:
def extract_text(text):
    soup = BeautifulSoup(text, 'lxml')
    txt = "".join([txt.text for txt in soup.find_all("p")])
    return txt

que_ans['body'] = que_ans['body'].apply(extract_text)
que_ans['question list'] = que_ans['title'] + que_ans['body']
que_ans.drop(columns=['title', 'body'], axis=1, inplace = True)

In [82]:
rel_que

,id,questions,score
0,48641350,How to merge two lists into a list of multiple...,0.799961
1,46362972,Merge list of lists in python 3,0.785947
2,1158128,Merge sorted lists in python,0.750189
3,44476206,how to merge two list having dict,0.749920
4,58971955,How to merge two dataframe?,0.709934
...,...,...,...
95,14008075,How to merge many to many relations from one d...,0.372127
96,55206146,Merge Pandas Dataframe with non-unique index w...,0.366641
97,58678560,How to merge similar data into a custom field ...,0.365843
98,50727548,How to merge multiple dataframes with the same...,0.354777


In [83]:
rel_ans

,answer,link,code,ans length,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,answer list
4728,You can either use update:output:Or loc:output:,NaN,update\r\nEmpty_DF.update(ROI_DF)\r\n\r\n ...,6,0,0,0,0,0,0,0,1,You can either use update:output:Or loc:output...
4729,"This is perfect case for DataFrame.update, whi...",https://pandas.pydata.org/pandas-docs/stable/r...,DataFrame.update\r\nEmpty_DF.update(ROI_DF)\r\...,44,1,0,0,1,0,0,0,0,"This is perfect case for DataFrame.update, whi..."
4730,In your case reindex_like,NaN,reindex_like\r\nyourdf=ROI_DF.reindex_like(Emp...,4,0,0,0,0,0,0,0,0,In your case reindex_likereindex_like\r\nyourd...
60589,You can put the different column into index an...,NaN,"pd.concat([plantsFrame.set_index(['plants']), ...",12,0,0,0,0,0,0,0,1,You can put the different column into index an...
27294,Your desired outputs are no longer really mode...,https://stackoverflow.com/a/35019122/12197595,NaN,46,0,1,0,0,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39444,Use this code :Ouptut:,NaN,"list1= [['user1', 186, 'Feb 2017, Apr 2017', 5...",4,0,0,0,1,0,0,1,0,"Use this code :Ouptut:list1= [['user1', 186, '..."
18975,This might help,NaN,"desiredlist = list(map(lambda y:[lst1,y],lst2)...",3,0,0,0,0,0,0,0,0,This might helpdesiredlist = list(map(lambda y...
18976,You could achieve your desired output with ite...,NaN,itertools.zip_longest\r\nfillvalue\r\n>>> from...,31,0,0,0,0,0,0,0,1,You could achieve your desired output with ite...
18977,"Or you can use use append, but you need to cre...",NaN,lst3 = []\r\nfor elem in lst2:\r\n theNew =...,16,1,0,0,0,0,0,0,1,"Or you can use use append, but you need to cre..."


In [84]:
que_ans

,id,score,question list
4728,58276538,2,How to merge/join empty dataframe with another...
4729,58276538,3,How to merge/join empty dataframe with another...
4730,58276538,1,How to merge/join empty dataframe with another...
60589,50727548,0,How to merge multiple dataframes with the same...
27294,58678560,0,How to merge similar data into a custom field ...
...,...,...,...
39444,46362972,1,Merge list of lists in python 3I have this 2 l...
18975,48641350,0,How to merge two lists into a list of multiple...
18976,48641350,1,How to merge two lists into a list of multiple...
18977,48641350,1,How to merge two lists into a list of multiple...


In [85]:
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [86]:
def text_process(text):
    if(type(text) != type(0.0)):
        text = text.lower()
        text = re.sub("[^a-z]", " ", text)
        text = text.strip()
        token = word_tokenize(text)
        text = [i for i in token if not i in stop_words]
        output = []
        for word in text:
            output.append(stemmer.stem(word))
        text = []
        for word in output:
            text.append(lemmatizer.lemmatize(word))
        text = " ".join(text)
    return text

In [87]:
que_ans['question list'] = que_ans['question list'].apply(text_process)
rel_ans['answer list'] = rel_ans['answer list'].apply(text_process)

In [88]:
data = rel_ans.join(que_ans)

In [119]:
def cosine_score(question, answer):
    if(type(answer) == type(0.0)):
        return 0
    else:
        sentences = [question, answer]
        vectorizer = CountVectorizer()
        vector = vectorizer.fit_transform(sentences)
        text1 = vector.toarray()[0].tolist()
        text2 = vector.toarray()[1].tolist()
        cosc = 1-distance.cosine(text1, text2)
        return cosc
    
def entropy(text, tfidf_dict):
    if(type(text) == type(0.0)):
        return 0
    else:
        token = word_tokenize(text)
        entropy = 0.0
        for word in token:
            try:
                entropy = entropy + tfidf_dict[word]
            except:
                entropy = entropy + 0.0
        return entropy
    
def query_sim_score(query, answer):
    score = 0
    if(type(answer) == type(0.0)):
        return 0
    else:
        query = query
        for q in query:
            if(q in answer):
                score =  score + 1
        score = score/len(query)
        return score

In [120]:
query = 'merge two lists in python'
query = text_process(query)

In [121]:
docs = data['answer list'].dropna().tolist()
cv = CountVectorizer(stop_words = stop_words)
word_count_vector = cv.fit_transform(docs)

In [122]:
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit_transform(word_count_vector)

<157x1345 sparse matrix of type '<class 'numpy.float64'>'
	with 4237 stored elements in Compressed Sparse Row format>

In [123]:
tfidf_array = tfidf_transformer.idf_
words = cv.get_feature_names()
tfidf_dict = {}
for i in range(0, len(tfidf_transformer.idf_)):
    tfidf_dict[words[i]] = tfidf_array[i]

In [124]:
def normalised_score():
    sc = ((score - min_score)/(max_score - min_score))
    return sc

In [143]:
def score(user_score, cosine_score, entropy_score, sim_score, min_user_score, max_user_score, min_cosine_score, max_cosine_score, min_entropy_score, max_entropy_score):
    if(min_user_score == max_user_score):
        user_score = user_score - min_user_score
    else:
        user_score = ((user_score - min_user_score)/(max_user_score - min_user_score))
        
    if(min_cosine_score == max_cosine_score):
        cosine_score = cosine_score - min_cosine_score
    else:
        cosine_score = ((cosine_score - min_cosine_score)/(max_cosine_score - min_cosine_score))
        
    if(min_entropy_score == max_entropy_score):
        entropy_score = entropy_score - min_entropy_score
    else:
        entropy_score = ((entropy_score - min_entropy_score)/(max_entropy_score - min_entropy_score))
    return (user_score + cosine_score + entropy_score + 2 * sim_score)

In [126]:
data

,answer,link,code,ans length,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,answer list,id,score,question list
4728,You can either use update:output:Or loc:output:,NaN,update\r\nEmpty_DF.update(ROI_DF)\r\n\r\n ...,6,0,0,0,0,0,0,0,1,either use updat output loc output updat empti...,58276538,2,merg join empti datafram anoth fill datafram e...
4729,"This is perfect case for DataFrame.update, whi...",https://pandas.pydata.org/pandas-docs/stable/r...,DataFrame.update\r\nEmpty_DF.update(ROI_DF)\r\...,44,1,0,0,1,0,0,0,0,perfect case datafram updat align indicesoutpu...,58276538,3,merg join empti datafram anoth fill datafram e...
4730,In your case reindex_like,NaN,reindex_like\r\nyourdf=ROI_DF.reindex_like(Emp...,4,0,0,0,0,0,0,0,0,case reindex likereindex like yourdf roi df re...,58276538,1,merg join empti datafram anoth fill datafram e...
60589,You can put the different column into index an...,NaN,"pd.concat([plantsFrame.set_index(['plants']), ...",12,0,0,0,0,0,0,0,1,put differ column index concat output pd conca...,50727548,0,merg multipl datafram number column one datafr...
27294,Your desired outputs are no longer really mode...,https://stackoverflow.com/a/35019122/12197595,NaN,46,0,1,0,0,1,0,0,1,NaN,58678560,0,merg similar data custom field use serial djan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39444,Use this code :Ouptut:,NaN,"list1= [['user1', 186, 'Feb 2017, Apr 2017', 5...",4,0,0,0,1,0,0,1,0,use code ouptut list user feb apr user mai use...,46362972,1,merg list list python list list desir output f...
18975,This might help,NaN,"desiredlist = list(map(lambda y:[lst1,y],lst2)...",3,0,0,0,0,0,0,0,0,might helpdesiredlist list map lambda lst lst,48641350,0,merg two list list multipl list given two list...
18976,You could achieve your desired output with ite...,NaN,itertools.zip_longest\r\nfillvalue\r\n>>> from...,31,0,0,0,0,0,0,0,1,could achiev desir output itertool zip longest...,48641350,1,merg two list list multipl list given two list...
18977,"Or you can use use append, but you need to cre...",NaN,lst3 = []\r\nfor elem in lst2:\r\n theNew =...,16,1,0,0,0,0,0,0,1,use use append need creat new copi lst lst ele...,48641350,1,merg two list list multipl list given two list...


In [144]:
for topic in topics:
    topic_data = data[data[topic]==1][['id', 'answer', 'link', 'code', 'score', 'answer list', 'question list']]
    topic_data.dropna()
    entropy_score = []
    topic_df = pd.DataFrame()
    for i in range(0, topic_data.shape[0]):
        cosc = cosine_score(topic_data.iloc[i]['question list'], topic_data.iloc[i]['answer list'])
        entro = entropy(topic_data.iloc[i]['answer list'], tfidf_dict)
        sim_score = query_sim_score(query, topic_data.iloc[i]['answer list'])
        topic_df = topic_df.append(pd.Series([topic_data.iloc[i]['answer'], topic_data.iloc[i]['link'], topic_data.iloc[i]['code'], topic_data.iloc[i]['score'], cosc, entro, sim_score]), ignore_index=True)
    if topic_df.shape[0]==0:
            continue
    else:
        topic_df.columns = ['answer', 'link', 'code', 'score', 'cosine score', 'entropy','sim score']
        min_user_score = topic_df['score'].min()
        max_user_score = topic_df['score'].max()
        min_cosine_score = topic_df['cosine score'].min()
        max_cosine_score = topic_df['cosine score'].max()
        min_entropy_score = topic_df['entropy'].min()
        max_entropy_score = topic_df['entropy'].max()
        scores = []
        for i in range(0, topic_df.shape[0]):
            scores.append(score(topic_df.iloc[i]['score'], topic_df.iloc[i]['cosine score'], topic_df.iloc[i]['entropy'], 
                                topic_df.iloc[i]['sim score'], min_user_score, max_user_score, min_cosine_score, max_cosine_score,
                                min_entropy_score, max_entropy_score))
        topic_df.drop(columns = ['score', 'cosine score', 'entropy'], inplace=True)
        topic_df['sc'] = scores
        topic_df.sort_values(by = 'sc', ascending=False, inplace=True, kind='quicksort')
        path = 'data/'+ topic +'.csv'
        topic_df = topic_df.head(15)
        topic_df.to_csv(path, index=False)